## Content Bases Recommdataion Using Amazon Fashion Dataset
- Problem Statemnet: "Recommend an Product to user based on Similarity of item/Content/Product"

In [ ]:
import pandas as pd 
directory = "/kaggle/input/amazon-fashion-products-2020/marketing_sample_for_amazon_com-amazon_fashion_products__20200201_20200430__30k_data.ldjson"
data=pd.read_json(directory, lines=True)

### Understanding Dataset


In [4]:
# print the number of feature and datapoint Dataset have?
print("Number of data point: ", data.shape[0],"\nNumber of feature :",data.shape[1])

Number of data point:  30000 
Number of feature : 33


In [5]:
#Features in Dataset
print(data.columns)


Index(['uniq_id', 'crawl_timestamp', 'asin', 'product_url', 'product_name',
       'image_urls__small', 'medium', 'large', 'browsenode', 'brand',
       'sales_price', 'weight', 'rating', 'sales_rank_in_parent_category',
       'sales_rank_in_child_category', 'delivery_type', 'meta_keywords',
       'amazon_prime__y_or_n', 'parent___child_category__all',
       'best_seller_tag__y_or_n', 'other_items_customers_buy',
       'product_details__k_v_pairs', 'discount_percentage', 'colour',
       'no__of_reviews', 'seller_name', 'seller_id', 'left_in_stock',
       'no__of_offers', 'no__of_sellers', 'technical_details__k_v_pairs',
       'formats___editions', 'name_of_author_for_books'],
      dtype='object')


#### Description of dataset
- Total Records Count : 132685 
- Date Range : 01st Feb 2020 - 30th Apr 2020  

#### Available Fields:
- uniq_id-- The unique ID of the product
- crawl_timestamp-- The time of the crawl to pull the data
- asin-- The ASIN of the product
- product_url-- The URL of the product
- product_name-- The name of the product
- image_urls__small-- The url of the images in small size
- medium-- The medium by which the product was seen
- large-- the size of the file
- browsenode
- seller_name-- the name of the seller of the product
- seller_id-- the ID of the seller of the product
- brand-- the brand of the product
- sales_price-- the price of the sale of the product
- discount_percentage-- the discount that was being offered on the product
- weight-- the weight of the product
- rating-- the rating of the product
- no__of_reviews-- The number of reviews that have been given to the product
- delivery_type-- the type of delivery the product will be delivered to the buyer
- meta_keywords-- the keywords used to search for the product
- amazon_prime__y_or_n-- If the buyer has an amazon prime membership or no
- best_seller_tag__y_or_n-- the tag of bestseller or no
- technical_details__k_v_pairs-- the repair was given or no

In [6]:
# select only the few feature from the dataset to perfom similartity/ content based recommendation 
data=data[['asin','product_url', 'product_name','sales_price','rating','meta_keywords','medium','brand']]

In [7]:
#print the Columns Using For this task
data.columns

Index(['asin', 'product_url', 'product_name', 'sales_price', 'rating',
       'meta_keywords', 'medium', 'brand'],
      dtype='object')

- asin-- The ASIN of the product
- product_url-- The URL of the product
- product_name-- The name of the product
- medium-- The medium by which the product was seen
- brand-- the brand of the product
- sales_price-- the price of the sale of the product
- rating-- the rating of the product
- meta_keywords-- the keywords used to search for the product

In [8]:
# see few instance of dataset 
data.head()

,asin,product_url,product_name,sales_price,rating,meta_keywords,medium,brand
0,B07STS2W9T,https://www.amazon.in/Facon-Kalamkari-Handbloc...,LA' Facon Cotton Kalamkari Handblock Saree Blo...,200.0,5.0,LA' Facon Cotton Kalamkari Handblock Saree Blo...,https://images-na.ssl-images-amazon.com/images...,LA' Facon
1,B07N6TD2WL,https://www.amazon.in/Sf-Jeans-Pantaloons-T-Sh...,Sf Jeans By Pantaloons Men's Plain Slim fit T-...,265.0,3.6,Sf Jeans By Pantaloons Men's Plain Slim fit T-...,https://images-na.ssl-images-amazon.com/images...,NaN
2,B07WJ6WPN1,https://www.amazon.in/LOVISTA-Traditional-Prin...,LOVISTA Cotton Gota Patti Tassel Traditional P...,660.0,3.5,LOVISTA Cotton Gota Patti Tassel Traditional P...,https://images-na.ssl-images-amazon.com/images...,LOVISTA
3,B07PYSF4WZ,https://www.amazon.in/People-Printed-Regular-T...,People Men's Printed Regular fit T-Shirt,195.0,3.0,"People Men's Printed Regular fit T-Shirt,People",https://images-na.ssl-images-amazon.com/images...,NaN
4,B082KXNM7X,https://www.amazon.in/Monte-Carlo-Cotton-Colla...,Monte Carlo Grey Solid Cotton Blend Polo Colla...,1914.0,5.0,Monte Carlo Grey Solid Cotton Blend Polo Colla...,https://images-na.ssl-images-amazon.com/images...,NaN


In [9]:
# Imformation Of Dataset 
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   asin           30000 non-null  object 
 1   product_url    30000 non-null  object 
 2   product_name   30000 non-null  object 
 3   sales_price    27110 non-null  float64
 4   rating         30000 non-null  float64
 5   meta_keywords  30000 non-null  object 
 6   medium         29998 non-null  object 
 7   brand          21857 non-null  object 
dtypes: float64(2), object(6)
memory usage: 1.8+ MB


### Deal With nullvalues in Differnt Feaature
- As we can See out of 30000 product 
    - 27110 sales price are only non null
    - 21857 brand have null values
    - only for two product we don't have any image
- Remove the Rows having null values

In [10]:
# remove the rows having null values
data.dropna(axis=0, inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19233 entries, 0 to 29999
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   asin           19233 non-null  object 
 1   product_url    19233 non-null  object 
 2   product_name   19233 non-null  object 
 3   sales_price    19233 non-null  float64
 4   rating         19233 non-null  float64
 5   meta_keywords  19233 non-null  object 
 6   medium         19233 non-null  object 
 7   brand          19233 non-null  object 
dtypes: float64(2), object(6)
memory usage: 1.3+ MB


### Analyse the Features

In [11]:
# Understand the feature "Brand"
print(data['brand'].describe())

# Max is an Top most frequent brand and it have around 5403 brands in it

count     19233
unique     5403
top         Max
freq        500
Name: brand, dtype: object


In [12]:
# See the Top 10 most frequent brand
from collections import Counter
counts=Counter(list(data['brand']))
counts.most_common(10)

[('Max', 500),
 ('Generic', 223),
 ('BIBA', 205),
 ('Mothercare', 156),
 ('Campus Sutra', 150),
 ('Soch', 149),
 ('nauti nati', 132),
 ('Ada', 125),
 ('GRITSTONES', 110),
 ('PrintOctopus', 102)]

In [13]:
# similar analysis of price feature as well
print(data['sales_price'].describe())
# AS we can see that minimum price of product is 39 and max 9988 rupee

count    19233.000000
mean       902.009351
std       1072.390707
min         39.000000
25%        365.000000
50%        559.000000
75%        899.000000
max       9988.000000
Name: sales_price, dtype: float64


In [14]:
#similarly for the "Product Name"
print(data['product_name'].describe())
# most common title have womens kurta

count                           19233
unique                          15949
top       BIBA Women's Straight Kurta
freq                               70
Name: product_name, dtype: object


## Finding and Removing Duplicate Values

In [15]:
# find number of product have the same product name
print( sum(data.duplicated('product_name')))

3284


In [16]:
 # Remove the product having Duplicate title
data = data.drop_duplicates(subset=['product_name'])

In [17]:
# remove the product with very few words in product name
d_data=data[data['product_name'].apply(lambda x: len(x.split())>4)]
d_data.shape
# around 3284 Data point is removed


(14933, 8)

In [18]:
#sort the data with their alphabetical order
d_data.sort_values('product_name',inplace=True,ascending=False)

/tmp/ipykernel_47/3414184361.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d_data.sort_values('product_name',inplace=True,ascending=False)


In [19]:
d_data['product_name'].head()
# there some differnt type of duplicate like only they are differ by only few words
# also they are different only by color or by keyword only
# we can remove them as well

22150                     zs cap house Owise Grey net topi
23619                           zs cap house Blue net topi
21756    zero by Hopscotch Girls' Cotton Art Print Casu...
26085    yazhi Men's Cotton T-Shirt - Be You - Dark Blu...
5523       women's manzella cable knit glove (pack of two)
Name: product_name, dtype: object

In [20]:
# PreProcssing the text 
# using nltk library to do it
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import nltk.stem
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from tqdm import tqdm
import re

In [21]:
# remove punctuation 
def remove_pun(sentence):
    cleaned=re.sub(r'[?|!|\|#|.|"|)|(|)|/|,|:|\'|-|$|+|~|;|-|_|@|>|<]',r'',sentence)
    return cleaned
#3. Remove word which are not english letter
def remove_alhpa(sentence):
    cleaned=("\S*\d\S*", "", sentence)
    return cleaned
# all english stop words
stopwords= set(stopwords.words('english'))
# some phases in written short word 
def decontracted(phrase):
    # specific short to long conversion
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general decontracted
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase
# stemming the words
sno=nltk.stem.SnowballStemmer('english')
# tqdm is for printing the status bar

product_name=[]
for sentence in tqdm(data['product_name'].values):
    #converting the pharases
    sentance = decontracted(sentence)
    #remove alha numeric
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    #convert every word in lower and take only if not in stop word and then snowball
    sentence = " ".join((sno.stem(e.lower())) for e in sentence.split() if e.lower() not in stopwords)
    # append the processed title to prodcut_name 
    product_name.append(remove_pun(sentence).strip())
data['product_name']=product_name

100%|██████████| 15949/15949 [00:03<00:00, 5198.78it/s]


In [22]:
print(data['product_name'].head())

0    la facon cotton kalamkari handblock sare blous...
2    lovista cotton gota patti tassel tradit print ...
5    forest club  gym wear  sport shorts short men ...
6    printoctopus graphic print t-shirt men chill t...
9                             peppermint synthet dress
Name: product_name, dtype: object


### Text Based Product Similarity using Sklearn and BOW , TF-IDF , Word2Vec , BERT

### Using BOW

In [23]:
from sklearn.feature_extraction.text import CountVectorizer
# Step 1 Convert the Text/Title using BOW
count_vect = CountVectorizer() #in scikit-learn that do as above description
count_vect.fit(data['product_name'])
final_counts = count_vect.transform(data['product_name'])
print("the type of count vectorizer ",type(final_counts))
print("the shape of out text BOW vectorizer ",final_counts.get_shape())
print("the number of unique words ", final_counts.get_shape()[1])

the type of count vectorizer  <class 'scipy.sparse._csr.csr_matrix'>
the shape of out text BOW vectorizer  (15949, 14330)
the number of unique words  14330


### Function to compute the product similarity using BOW as text to vector

In [111]:
# for given a product title find 'x' similar Product 
# Algorithm
# find the pairwise distance bewteen given point and all other point using eculidean distance
# sort them using np.argsort
#pick the top 'x' title having the smallest distance 
# print them as a output the Product that can be Recommmended to user based on product
# similarity using BOW as a text to vector technique and eculidean distance as the similarity
from sklearn.metrics import pairwise_distances
import numpy as np
from IPython.display import Image, display
def RecommenderSystems(p_id,numProduct,vectors):
    """
    p_id is the index from data that  is preprocessed
    numProduct of wanted to recommeded
    """
    # Compute the Pairwise Distances
    dist=pairwise_distances(vectors,vectors[p_id].reshape(1,-1))
    # sort them based the distance computed
    indices=np.argsort(dist.flatten())[0:numProduct]
    # this will store distances between the product_titles
    p_dist=np.sort(dist.flatten())[0:numProduct]
    # take product details for these indices
    similar_product=list(data.index[indices])
    for i in range(len(indices)):
        # generate the heatmap as well
        display(Image(url=data['medium'].loc[similar_product[i]],width=100, height=100))
        print('ASIN :',data['asin'].loc[similar_product[i]])
        print('Brand :',data['brand'].loc[similar_product[i]])
        print('Product Name :',data['product_name'].loc[similar_product[i]])
        print('Distance :',p_dist[i])
        print('_'*60)


### Поиск по текстовому запросу пользователя


In [ ]:
def preprocess_query(query):
    """
    Предобработка текстового запроса пользователя (та же логика, что и для названий товаров)
    """
    sentance = decontracted(query)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    sentence = " ".join((sno.stem(e.lower())) for e in sentance.split() if e.lower() not in stopwords)
    return remove_pun(sentence).strip()


In [ ]:
def SearchByQueryBOW(query, numProduct, vectorizer, vectors):
    """
    Поиск товаров по текстовому запросу используя BOW
    query - текстовый запрос пользователя
    numProduct - количество товаров для возврата
    vectorizer - обученный CountVectorizer
    vectors - векторы всех товаров (BOW)
    """
    query_processed = preprocess_query(query)
    query_vec = vectorizer.transform([query_processed])
    
    dist = pairwise_distances(vectors, query_vec)
    indices = np.argsort(dist.flatten())[0:numProduct]
    p_dist = np.sort(dist.flatten())[0:numProduct]
    
    similar_product = list(data.index[indices])
    print(f"Запрос: '{query}' -> обработан: '{query_processed}'")
    print(f"Найдено товаров: {numProduct}\n")
    
    for i in range(len(indices)):
        display(Image(url=data['medium'].loc[similar_product[i]], width=100, height=100))
        print('ASIN :', data['asin'].loc[similar_product[i]])
        print('Brand :', data['brand'].loc[similar_product[i]])
        print('Product Name :', data['product_name'].loc[similar_product[i]])
        print('Distance :', p_dist[i])
        print('_'*60)


In [ ]:
def SearchByQueryTFIDF(query, numProduct, vectorizer, vectors):
    """
    Поиск товаров по текстовому запросу используя TF-IDF
    query - текстовый запрос пользователя
    numProduct - количество товаров для возврата
    vectorizer - обученный TfidfVectorizer
    vectors - векторы всех товаров (TF-IDF)
    """
    query_processed = preprocess_query(query)
    query_vec = vectorizer.transform([query_processed])
    
    dist = pairwise_distances(vectors, query_vec)
    indices = np.argsort(dist.flatten())[0:numProduct]
    p_dist = np.sort(dist.flatten())[0:numProduct]
    
    similar_product = list(data.index[indices])
    print(f"Запрос: '{query}' -> обработан: '{query_processed}'")
    print(f"Найдено товаров: {numProduct}\n")
    
    for i in range(len(indices)):
        display(Image(url=data['medium'].loc[similar_product[i]], width=100, height=100))
        print('ASIN :', data['asin'].loc[similar_product[i]])
        print('Brand :', data['brand'].loc[similar_product[i]])
        print('Product Name :', data['product_name'].loc[similar_product[i]])
        print('Distance :', p_dist[i])
        print('_'*60)


In [ ]:
def SearchByQueryWord2Vec(query, numProduct, w2v_model, w2v_words, vectors):
    """
    Поиск товаров по текстовому запросу используя Word2Vec
    query - текстовый запрос пользователя
    numProduct - количество товаров для возврата
    w2v_model - обученная модель Word2Vec
    w2v_words - список слов в модели
    vectors - векторы всех товаров (Word2Vec)
    """
    query_processed = preprocess_query(query)
    
    query_vec = np.zeros(300)
    cnt_words = 0
    for word in query_processed.split():
        if word in w2v_words:
            vec = w2v_model[word]
            query_vec += vec
            cnt_words += 1
    
    if cnt_words == 0:
        print(f"Запрос: '{query}' -> обработан: '{query_processed}'")
        print("Ошибка: ни одно слово из запроса не найдено в модели Word2Vec")
        return
    
    query_vec /= cnt_words
    query_vec = query_vec.reshape(1, -1)
    
    dist = pairwise_distances(vectors, query_vec)
    indices = np.argsort(dist.flatten())[0:numProduct]
    p_dist = np.sort(dist.flatten())[0:numProduct]
    
    similar_product = list(data.index[indices])
    print(f"Запрос: '{query}' -> обработан: '{query_processed}'")
    print(f"Найдено товаров: {numProduct}\n")
    
    for i in range(len(indices)):
        display(Image(url=data['medium'].loc[similar_product[i]], width=100, height=100))
        print('ASIN :', data['asin'].loc[similar_product[i]])
        print('Brand :', data['brand'].loc[similar_product[i]])
        print('Product Name :', data['product_name'].loc[similar_product[i]])
        print('Distance :', p_dist[i])
        print('_'*60)


In [ ]:
def SearchByQueryTFIDFWord2Vec(query, numProduct, w2v_model, w2v_words, tfidf_dict, tfidf_feat, vectors):
    """
    Поиск товаров по текстовому запросу используя TF-IDF weighted Word2Vec
    query - текстовый запрос пользователя
    numProduct - количество товаров для возврата
    w2v_model - обученная модель Word2Vec
    w2v_words - список слов в модели
    tfidf_dict - словарь IDF значений
    tfidf_feat - список признаков TF-IDF
    vectors - векторы всех товаров (TF-IDF weighted Word2Vec)
    """
    query_processed = preprocess_query(query)
    
    query_vec = np.zeros(300)
    weight_sum = 0
    query_words = query_processed.split()
    
    for word in query_words:
        if word in w2v_words and word in tfidf_feat:
            vec = w2v_model[word]
            tf_idf = tfidf_dict[word] * (query_words.count(word) / len(query_words))
            query_vec += (vec * tf_idf)
            weight_sum += tf_idf
    
    if weight_sum == 0:
        print(f"Запрос: '{query}' -> обработан: '{query_processed}'")
        print("Ошибка: не удалось создать вектор для запроса")
        return
    
    query_vec /= weight_sum
    query_vec = query_vec.reshape(1, -1)
    
    dist = pairwise_distances(vectors, query_vec)
    indices = np.argsort(dist.flatten())[0:numProduct]
    p_dist = np.sort(dist.flatten())[0:numProduct]
    
    similar_product = list(data.index[indices])
    print(f"Запрос: '{query}' -> обработан: '{query_processed}'")
    print(f"Найдено товаров: {numProduct}\n")
    
    for i in range(len(indices)):
        display(Image(url=data['medium'].loc[similar_product[i]], width=100, height=100))
        print('ASIN :', data['asin'].loc[similar_product[i]])
        print('Brand :', data['brand'].loc[similar_product[i]])
        print('Product Name :', data['product_name'].loc[similar_product[i]])
        print('Distance :', p_dist[i])
        print('_'*60)


In [ ]:
def SearchByQueryWeighted(query, numProduct, w2v_model, w2v_words, tfidf_dict, tfidf_feat, 
                          text_vectors, brand_vectors, category_weights):
    """
    Поиск товаров по текстовому запросу с использованием взвешенных признаков
    query - текстовый запрос пользователя
    numProduct - количество товаров для возврата
    text_vectors - векторы текстов (TF-IDF weighted Word2Vec)
    brand_vectors - векторы брендов (One-Hot)
    category_weights - веса для каждого признака [текст, бренд]
    """
    query_processed = preprocess_query(query)
    
    query_text_vec = np.zeros(300)
    weight_sum = 0
    query_words = query_processed.split()
    
    for word in query_words:
        if word in w2v_words and word in tfidf_feat:
            vec = w2v_model[word]
            tf_idf = tfidf_dict[word] * (query_words.count(word) / len(query_words))
            query_text_vec += (vec * tf_idf)
            weight_sum += tf_idf
    
    if weight_sum == 0:
        print(f"Запрос: '{query}' -> обработан: '{query_processed}'")
        print("Ошибка: не удалось создать вектор для запроса")
        return
    
    query_text_vec /= weight_sum
    query_text_vec = query_text_vec * category_weights[0]
    
    query_brand_vec = np.zeros(brand_vectors.shape[1]) * category_weights[1]
    
    query_vec = np.concatenate([query_text_vec, query_brand_vec]).reshape(1, -1)
    
    weighted_vectors = []
    for i in range(len([text_vectors, brand_vectors])):
        vec = np.array([text_vectors, brand_vectors][i]) * category_weights[i]
        weighted_vectors.append(vec)
    all_vectors = np.concatenate(weighted_vectors, axis=1)
    
    dist = pairwise_distances(all_vectors, query_vec)
    indices = np.argsort(dist.flatten())[0:numProduct]
    p_dist = np.sort(dist.flatten())[0:numProduct]
    
    similar_product = list(data.index[indices])
    print(f"Запрос: '{query}' -> обработан: '{query_processed}'")
    print(f"Найдено товаров: {numProduct}\n")
    
    for i in range(len(indices)):
        display(Image(url=data['medium'].loc[similar_product[i]], width=100, height=100))
        print('ASIN :', data['asin'].loc[similar_product[i]])
        print('Brand :', data['brand'].loc[similar_product[i]])
        print('Product Name :', data['product_name'].loc[similar_product[i]])
        print('Distance :', p_dist[i])
        print('_'*60)


### Примеры использования поиска по запросу


In [ ]:
# Пример 1: Поиск по BOW
SearchByQueryBOW("women cotton dress", 5, count_vect, final_counts)


In [ ]:
# Пример 2: Поиск по TF-IDF
SearchByQueryTFIDF("men t-shirt", 5, tifd, tf)


In [ ]:
# Пример 3: Поиск по Word2Vec
SearchByQueryWord2Vec("baseball cap", 5, w2v_model, w2v_words, pv)


In [ ]:
# Пример 4: Поиск по TF-IDF weighted Word2Vec (рекомендуется)
SearchByQueryTFIDFWord2Vec("kurta women", 5, w2v_model, w2v_words, dictionary, tfidf_feat, tfidf_sent_vectors)


In [ ]:
# Пример 5: Поиск с учетом текста и бренда (взвешенный)
SearchByQueryWeighted("handloom saree", 5, w2v_model, w2v_words, dictionary, tfidf_feat, 
                      tfidf_sent_vectors, brand_vec.values, [1, 0.5])


In [ ]:
# Скачивание необходимых ресурсов NLTK
import nltk

try:
    nltk.data.find('corpora/stopwords')
    print("Stopwords уже загружены")
except LookupError:
    print("Скачиваю stopwords...")
    nltk.download('stopwords', quiet=True)
    print("Stopwords загружены!")

try:
    nltk.data.find('tokenizers/punkt')
    print("Punkt tokenizer уже загружен")
except LookupError:
    print("Скачиваю punkt tokenizer...")
    nltk.download('punkt', quiet=True)
    print("Punkt tokenizer загружен!")

print("Все необходимые ресурсы NLTK готовы!")

In [ ]:
# Проверка наличия необходимых переменных и их создание при необходимости
try:
    # Проверяем наличие tifd и tf
    if 'tifd' not in globals() or 'tf' not in globals():
        print("Создаю TF-IDF векторизатор...")
        from sklearn.feature_extraction.text import TfidfVectorizer
        tifd = TfidfVectorizer(ngram_range=(1,2), min_df=10)
        tf = tifd.fit_transform(data['product_name'])
        print(f"TF-IDF векторизатор создан. Размерность: {tf.get_shape()}")
    else:
        print("TF-IDF векторизатор уже существует")
    
    # Проверяем наличие функции preprocess_query и создаем простую версию, если её нет
    if 'preprocess_query' not in globals():
        print("Создаю упрощенную версию preprocess_query...")
        import re
        import nltk
        from nltk.corpus import stopwords
        import nltk.stem
        
        try:
            nltk.data.find('corpora/stopwords')
        except LookupError:
            nltk.download('stopwords', quiet=True)
        
        stopwords_set = set(stopwords.words('english'))
        sno = nltk.stem.SnowballStemmer('english')
        
        def decontracted(phrase):
            phrase = re.sub(r"won't", "will not", phrase)
            phrase = re.sub(r"can\'t", "can not", phrase)
            phrase = re.sub(r"n\'t", " not", phrase)
            phrase = re.sub(r"\'re", " are", phrase)
            phrase = re.sub(r"\'s", " is", phrase)
            phrase = re.sub(r"\'d", " would", phrase)
            phrase = re.sub(r"\'ll", " will", phrase)
            phrase = re.sub(r"\'t", " not", phrase)
            phrase = re.sub(r"\'ve", " have", phrase)
            phrase = re.sub(r"\'m", " am", phrase)
            return phrase
        
        def remove_pun(sentence):
            cleaned = re.sub(r'[?|!|\|#|.|"|)|(|)|/|,|:|\'|-|$|+|~|;|-|_|@|>|<]', r'', sentence)
            return cleaned
        
        def preprocess_query(query):
            sentance = decontracted(query)
            sentance = re.sub("\S*\d\S*", "", sentance).strip()
            sentance = re.sub('[^A-Za-z]+', ' ', sentance)
            sentence = " ".join((sno.stem(e.lower())) for e in sentance.split() if e.lower() not in stopwords_set)
            return remove_pun(sentence).strip()
        
        print("Функция preprocess_query создана")
    else:
        print("Функция preprocess_query найдена")
        
except NameError as e:
    print(f"Ошибка: {e}")
    print("Убедитесь, что переменная 'data' определена (загружены данные)")

from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def generate_feed_tfidf(query, n_products, vectorizer, product_vectors, data_df, rows=2):
    """
    Генерирует выдачу товаров на основе TF-IDF эмбеддингов
    
    Параметры:
    -----------
    query : str
        Текстовый запрос пользователя
    n_products : int
        Количество товаров для выдачи (должно быть кратно rows*2)
    vectorizer : TfidfVectorizer
        Обученный TF-IDF векторизатор
    product_vectors : scipy.sparse matrix
        TF-IDF векторы всех товаров
    data_df : pandas.DataFrame
        DataFrame с данными о товарах
    rows : int
        Количество строк в выдаче (по умолчанию 2, итого rows*2 товаров)
    
    Возвращает:
    -----------
    results : list of dict
        Список словарей с информацией о товарах и коэффициентом релевантности
    """
    # Предобработка запроса
    query_processed = preprocess_query(query)
    
    # Векторизация запроса
    query_vec = vectorizer.transform([query_processed])
    
    # Вычисление косинусного сходства между запросом и всеми товарами
    similarities = cosine_similarity(product_vectors, query_vec).flatten()
    
    # Нормализация в диапазон [0, 1]
    # Косинусное сходство уже в диапазоне [-1, 1], нормализуем в [0, 1]
    min_sim = similarities.min()
    max_sim = similarities.max()
    if max_sim - min_sim > 0:
        relevance_scores = (similarities - min_sim) / (max_sim - min_sim)
    else:
        relevance_scores = np.ones_like(similarities)
    
    # Выбор топ-n товаров
    top_indices = np.argsort(relevance_scores)[::-1][:n_products]
    
    # Формирование результатов (только id, название, изображение)
    results = []
    for idx in top_indices:
        product_idx = data_df.index[idx]
        results.append({
            'id': data_df.loc[product_idx, 'asin'],
            'product_name': data_df.loc[product_idx, 'product_name'],
            'image_url': data_df.loc[product_idx, 'medium'],
            'relevance_score': float(relevance_scores[idx]),
            'vector_index': int(idx)  # для вычисления переходов
        })
    
    return results

def display_feed_grid(results, rows=2, cols=2):
    """
    Отображает выдачу товаров в виде сетки l*2
    
    Параметры:
    -----------
    results : list of dict
        Список результатов от generate_feed_tfidf
    rows : int
        Количество строк
    cols : int
        Количество колонок (обычно 2)
    """
    from IPython.display import display, HTML, Image
    import pandas as pd
    
    print(f"Выдача товаров ({rows} строки x {cols} колонки):\n")
    print("=" * 80)
    
    for i in range(0, len(results), cols):
        row_products = results[i:i+cols]
        print(f"\nСтрока {i//cols + 1}:")
        print("-" * 80)
        
        for j, product in enumerate(row_products):
            print(f"\nТовар {i+j+1} (колонка {j+1}):")
            print(f"  ID: {product['id']}")
            print(f"  Название: {product['product_name']}")
            if 'purchase_probability' in product:
                print(f"  Вероятность покупки: {product['purchase_probability']:.4f}")
            if product['image_url']:
                try:
                    display(Image(url=product['image_url'], width=150, height=150))
                except:
                    print(f"  Изображение: {product['image_url']}")
            print()
        
        if i + cols < len(results):
            print("\n" + "=" * 80)
    
    # Сводная таблица
    print("\n" + "=" * 80)
    print("Сводная таблица результатов:")
    print("=" * 80)
    df_results = pd.DataFrame(results)
    columns_to_show = ['id', 'product_name', 'image_url']
    if 'purchase_probability' in df_results.columns:
        columns_to_show.append('purchase_probability')
    display(df_results[columns_to_show])


In [ ]:
# Пример использования модели
query = "women cotton dress"
n_products = 4  # 2 строки * 2 колонки
rows = 2

results = generate_feed_tfidf(query, n_products, tifd, tf, data, rows=rows)
display_feed_grid(results, rows=rows, cols=2)


In [ ]:
def calculate_transition_probabilities(results, data_df, rows=2, cols=2, image_model=None):
    """
    Вычисляет вероятности переходов между соседними товарами в сетке на основе схожести эмбеддингов изображений
    
    Параметры:
    -----------
    results : list of dict
        Список товаров из generate_feed_tfidf (содержит id, product_name, image_url)
    data_df : pandas.DataFrame
        DataFrame с данными о товарах
    rows : int
        Количество строк в сетке
    cols : int
        Количество колонок в сетке
    image_model : tensorflow model, optional
        Модель VGG16 для извлечения эмбеддингов изображений. Если None, попытается загрузить.
    
    Возвращает:
    -----------
    transitions : dict
        Словарь с вероятностями переходов:
        - 'down': вероятности переходов вниз (для каждого товара)
        - 'left': вероятности переходов налево
        - 'right': вероятности переходов направо
    """
    from sklearn.metrics.pairwise import cosine_similarity
    import tensorflow as tf
    from tensorflow.keras.applications import VGG16
    from tensorflow.keras.preprocessing import image
    from tensorflow.keras.applications.vgg16 import preprocess_input
    from tensorflow.keras.utils import get_file
    
    # Загружаем модель VGG16 если не передана
    if image_model is None:
        try:
            image_model = VGG16(weights='imagenet', include_top=False)
            print("Модель VGG16 загружена")
        except:
            print("Ошибка загрузки VGG16. Используем эмбеддинги названий как fallback.")
            image_model = None
    
    def extract_image_embedding(img_url, item_id):
        """Извлекает эмбеддинг изображения"""
        if image_model is None:
            return None
        try:
            # Обрабатываем URL - может быть несколько через |
            if '|' in str(img_url):
                img_url = str(img_url).split('|')[0]
            
            # Используем уникальное имя файла для каждого товара
            import hashlib
            url_hash = hashlib.md5(str(img_url).encode()).hexdigest()[:8]
            temp_filename = f'temp_image_{item_id}_{url_hash}.jpg'
            
            img_path = get_file(temp_filename, origin=img_url)
            img = image.load_img(img_path, target_size=(224, 224))
            img_array = image.img_to_array(img)
            img_array = np.expand_dims(img_array, axis=0)
            img_array = preprocess_input(img_array)
            features = image_model.predict(img_array, verbose=0)
            embedding = features.flatten()
            
            # Проверяем, что эмбеддинг не нулевой
            if np.allclose(embedding, 0):
                print(f"    ВНИМАНИЕ: Эмбеддинг нулевой для {item_id}")
                return None
            
            return embedding
        except Exception as e:
            print(f"    Ошибка обработки изображения {item_id} ({img_url[:50]}...): {e}")
            return None
    
    # Извлекаем эмбеддинги изображений для всех товаров в выдаче
    print("Извлечение эмбеддингов изображений...")
    print(f"URL изображений:")
    for i, item in enumerate(results):
        print(f"  Товар {i+1} ({item['id']}): {str(item['image_url'])[:100]}...")
    
    image_embeddings = []
    for i, item in enumerate(results):
        embedding = extract_image_embedding(item['image_url'], item['id'])
        if embedding is not None:
            image_embeddings.append(embedding)
            print(f"  Товар {i+1} ({item['id']}): эмбеддинг извлечен, размер={len(embedding)}, norm={np.linalg.norm(embedding):.4f}, min={embedding.min():.4f}, max={embedding.max():.4f}")
        else:
            # Fallback: используем случайный вектор, чтобы они были разные
            import random
            random.seed(hash(item['id']) % 1000)
            fallback_emb = np.random.rand(25088) * 0.01  # маленькие случайные значения
            image_embeddings.append(fallback_emb)
            print(f"  Товар {i+1} ({item['id']}): ошибка извлечения, используется случайный вектор")
    
    image_embeddings = np.array(image_embeddings)
    
    # Проверяем, что эмбеддинги разные
    print(f"\nПроверка эмбеддингов:")
    print(f"  Размер массива: {image_embeddings.shape}")
    norms = [np.linalg.norm(emb) for emb in image_embeddings]
    print(f"  Норма каждого эмбеддинга: {norms}")
    
    # Проверяем, не все ли эмбеддинги одинаковые
    if len(image_embeddings) > 1:
        first_emb = image_embeddings[0]
        all_same = True
        differences = []
        for i, emb in enumerate(image_embeddings[1:], 1):
            diff = np.linalg.norm(emb - first_emb)
            differences.append(diff)
            if not np.allclose(emb, first_emb, atol=1e-6):
                all_same = False
        
        print(f"  Все эмбеддинги одинаковые: {all_same}")
        print(f"  Различия между эмбеддингами: {differences}")
        if all_same:
            print("  ВНИМАНИЕ: Все эмбеддинги одинаковые! Возможно, изображения не загружаются или кэшируются.")
        else:
            print(f"  Эмбеддинги разные. Минимальная разница: {min(differences):.6f}, максимальная: {max(differences):.6f}")
    
    # Вычисляем матрицу схожести между эмбеддингами изображений
    similarity_matrix = cosine_similarity(image_embeddings)
    print(f"\nМатрица схожести вычислена. Размер: {similarity_matrix.shape}")
    print(f"Диапазон схожести: min={similarity_matrix.min():.6f}, max={similarity_matrix.max():.6f}")
    print(f"Диагональ (схожесть товара с самим собой): {np.diag(similarity_matrix)}")
    print(f"Полная матрица схожести:\n{similarity_matrix}")
    
    # Косинусное сходство уже в диапазоне [-1, 1], но для изображений обычно [0, 1]
    # Нормализуем в [0, 1] если нужно
    min_sim_global = similarity_matrix.min()
    max_sim_global = similarity_matrix.max()
    
    print(f"\nНормализация:")
    print(f"  min_sim_global={min_sim_global:.6f}, max_sim_global={max_sim_global:.6f}")
    
    # Если все значения одинаковые (например, все 1.0), это проблема
    if abs(max_sim_global - min_sim_global) < 1e-6:
        print("  ВНИМАНИЕ: Все схожести одинаковые! Используем схожесть напрямую без нормализации.")
        similarity_matrix_normalized = similarity_matrix
    elif min_sim_global < 0:
        similarity_matrix_normalized = (similarity_matrix - min_sim_global) / (max_sim_global - min_sim_global)
        print(f"  Нормализация с отрицательными значениями применена")
    else:
        # Если все значения положительные, нормализуем относительно максимума
        if max_sim_global > 0:
            similarity_matrix_normalized = similarity_matrix / max_sim_global
            print(f"  Нормализация делением на максимум: {max_sim_global:.6f}")
        else:
            similarity_matrix_normalized = similarity_matrix
            print(f"  Нормализация не применена (max=0)")
    
    print(f"После нормализации: min={similarity_matrix_normalized.min():.6f}, max={similarity_matrix_normalized.max():.6f}")
    print(f"Нормализованная матрица:\n{similarity_matrix_normalized}")
    
    transitions = {
        'down': {},  # переход вниз: (row, col) -> (row+1, col)
        'left': {},  # переход налево: (row, col) -> (row, col-1)
        'right': {}  # переход направо: (row, col) -> (row, col+1)
    }
    
    # Проходим по сетке и вычисляем вероятности переходов
    for row in range(rows):
        for col in range(cols):
            idx = row * cols + col
            if idx >= len(results):
                continue
            
            current_similarities = similarity_matrix[idx]
            
            # Переход вниз
            if row < rows - 1:
                down_idx = (row + 1) * cols + col
                if down_idx < len(results):
                    # Вероятность = нормализованная схожесть эмбеддингов
                    similarity = similarity_matrix[idx, down_idx]
                    prob = float(similarity_matrix_normalized[idx, down_idx])
                    transitions['down'][(row, col)] = {
                        'to': (row + 1, col),
                        'probability': prob,
                        'similarity': float(similarity),
                        'from_id': results[idx]['id'],
                        'to_id': results[down_idx]['id']
                    }
            
            # Переход налево
            if col > 0:
                left_idx = row * cols + (col - 1)
                if left_idx < len(results):
                    similarity = similarity_matrix[idx, left_idx]
                    prob = float(similarity_matrix_normalized[idx, left_idx])
                    transitions['left'][(row, col)] = {
                        'to': (row, col - 1),
                        'probability': prob,
                        'similarity': float(similarity),
                        'from_id': results[idx]['id'],
                        'to_id': results[left_idx]['id']
                    }
            
            # Переход направо
            if col < cols - 1:
                right_idx = row * cols + (col + 1)
                if right_idx < len(results):
                    similarity = similarity_matrix[idx, right_idx]
                    prob = float(similarity_matrix_normalized[idx, right_idx])
                    transitions['right'][(row, col)] = {
                        'to': (row, col + 1),
                        'probability': prob,
                        'similarity': float(similarity),
                        'from_id': results[idx]['id'],
                        'to_id': results[right_idx]['id']
                    }
    
    return transitions

def display_transition_probabilities(transitions, results, rows=2, cols=2):
    """
    Отображает вероятности переходов в удобном виде
    """
    print("=" * 80)
    print("ВЕРОЯТНОСТИ ПЕРЕХОДОВ МЕЖДУ ТОВАРАМИ")
    print("=" * 80)
    
    # Создаем сетку для визуализации
    grid = [[None for _ in range(cols)] for _ in range(rows)]
    for i, item in enumerate(results):
        row = i // cols
        col = i % cols
        if row < rows and col < cols:
            grid[row][col] = item
    
    print("\nСетка товаров:")
    print("-" * 80)
    for row in range(rows):
        for col in range(cols):
            if grid[row][col]:
                print(f"({row}, {col}): {grid[row][col]['id']} - {grid[row][col]['product_name'][:50]}...")
    
    print("\n" + "=" * 80)
    print("ПЕРЕХОДЫ ВНИЗ:")
    print("=" * 80)
    for (from_row, from_col), info in transitions['down'].items():
        print(f"Из ({from_row}, {from_col}) -> ({info['to'][0]}, {info['to'][1]}):")
        print(f"  Вероятность: {info['probability']:.4f}")
        print(f"  Схожесть эмбеддингов: {info['similarity']:.4f}")
        print(f"  От: {info['from_id']}")
        print(f"  К: {info['to_id']}")
        print()
    
    print("=" * 80)
    print("ПЕРЕХОДЫ НАЛЕВО:")
    print("=" * 80)
    for (from_row, from_col), info in transitions['left'].items():
        print(f"Из ({from_row}, {from_col}) -> ({info['to'][0]}, {info['to'][1]}):")
        print(f"  Вероятность: {info['probability']:.4f}")
        print(f"  Схожесть эмбеддингов: {info['similarity']:.4f}")
        print(f"  От: {info['from_id']}")
        print(f"  К: {info['to_id']}")
        print()
    
    print("=" * 80)
    print("ПЕРЕХОДЫ НАПРАВО:")
    print("=" * 80)
    for (from_row, from_col), info in transitions['right'].items():
        print(f"Из ({from_row}, {from_col}) -> ({info['to'][0]}, {info['to'][1]}):")
        print(f"  Вероятность: {info['probability']:.4f}")
        print(f"  Схожесть эмбеддингов: {info['similarity']:.4f}")
        print(f"  От: {info['from_id']}")
        print(f"  К: {info['to_id']}")
        print()


In [ ]:
# Пример: вычисление вероятностей переходов
query = "women cotton dress"
n_products = 4
rows = 2
cols = 2

# Генерируем выдачу
results = generate_feed_tfidf(query, n_products, tifd, tf, data, rows=rows)

# Вычисляем вероятности переходов на основе эмбеддингов изображений
transitions = calculate_transition_probabilities(results, data, rows=rows, cols=cols)

# Отображаем результаты
display_transition_probabilities(transitions, results, rows=rows, cols=cols)


### Вычисление вероятности покупки товара на основе схожести запроса и названия


In [ ]:
# Установка sentence-transformers (если не установлен)
try:
    import sentence_transformers
except ImportError:
    print("Установка sentence-transformers...")
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "sentence-transformers", "-q"])
    print("sentence-transformers установлен!")


In [ ]:
# Переопределяем функции calculate_purchase_probability и get_embedding_model
# Глобальная переменная для кэширования модели эмбеддингов
_cached_embedding_model = None

def get_embedding_model():
    """
    Получает готовую модель для эмбеддингов предложений
    Использует sentence-transformers (лучший вариант) или Universal Sentence Encoder
    """
    global _cached_embedding_model
    
    if _cached_embedding_model is not None:
        return _cached_embedding_model
    
    # Пробуем sentence-transformers (лучший вариант)
    try:
        from sentence_transformers import SentenceTransformer
        print("Загрузка модели sentence-transformers...")
        _cached_embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
        print("Модель sentence-transformers загружена (all-MiniLM-L6-v2)")
        return _cached_embedding_model
    except ImportError:
        print("sentence-transformers не установлен. Пропускаем...")
    except Exception as e:
        print(f"Ошибка загрузки sentence-transformers: {e}")
        print("Пробуем другие модели...")
    
    # Fallback: пробуем Universal Sentence Encoder через tensorflow_hub
    try:
        import tensorflow_hub as hub
        print("Загрузка Universal Sentence Encoder...")
        _cached_embedding_model = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
        print("Universal Sentence Encoder загружен")
        return _cached_embedding_model
    except ImportError:
        print("tensorflow_hub не установлен. Установите: pip install tensorflow-hub")
    except Exception as e:
        print(f"Ошибка загрузки Universal Sentence Encoder: {e}")
    
    # Последний fallback: пробуем загрузить предобученный Word2Vec
    try:
        from gensim.models import KeyedVectors
        print("Попытка загрузки предобученной модели Word2Vec...")
        # Проверяем, существует ли файл
        import os
        w2v_path = '/kaggle/input/gensimmodel/GoogleNews-vectors-negative300.bin'
        if os.path.exists(w2v_path):
            _cached_embedding_model = KeyedVectors.load_word2vec_format(w2v_path, binary=True)
            print("Предобученная модель Word2Vec загружена")
            return _cached_embedding_model
        else:
            # Пробуем использовать уже загруженную модель, если она есть в глобальной области
            if 'w2v_model' in globals():
                print("Используем уже загруженную модель Word2Vec из глобальной области")
                _cached_embedding_model = globals()['w2v_model']
                return _cached_embedding_model
            else:
                print(f"Файл Word2Vec не найден: {w2v_path}")
    except Exception as e:
        print(f"Ошибка загрузки Word2Vec: {e}")
        # Пробуем использовать уже загруженную модель, если она есть
        if 'w2v_model' in globals():
            print("Используем уже загруженную модель Word2Vec из глобальной области")
            _cached_embedding_model = globals()['w2v_model']
            return _cached_embedding_model
    
    print("Не удалось загрузить ни одну модель эмбеддингов. Будет использован TF-IDF fallback.")
    return None

def calculate_purchase_probability(query, product_name, embedding_model=None):
    """
    Вычисляет вероятность покупки товара на основе схожести текста запроса и названия товара
    Использует готовую модель для эмбеддингов (sentence-transformers, Universal Sentence Encoder или Word2Vec)
    
    Параметры:
    -----------
    query : str
        Текстовый запрос пользователя
    product_name : str
        Название товара
    embedding_model : optional
        Модель для эмбеддингов. Если None, попытается загрузить готовую модель.
    
    Возвращает:
    -----------
    probability : float
        Вероятность покупки в диапазоне [0, 1]
    similarity : float
        Схожесть эмбеддингов (до нормализации)
    """
    from sklearn.metrics.pairwise import cosine_similarity
    import numpy as np
    
    # Получаем модель эмбеддингов
    if embedding_model is None:
        embedding_model = get_embedding_model()
        if embedding_model is None:
            # Fallback: используем простой TF-IDF
            try:
                return calculate_purchase_probability_tfidf(query, product_name, None, None, data if 'data' in globals() else None)
            except Exception as e:
                print(f"Ошибка в TF-IDF fallback: {e}")
                return 0.5, 0.0  # Возвращаем среднее значение
    
    # Определяем тип модели и получаем эмбеддинги
    try:
        # sentence-transformers
        if hasattr(embedding_model, 'encode'):
            query_embedding = embedding_model.encode([query])[0]
            product_embedding = embedding_model.encode([product_name])[0]
        
        # Universal Sentence Encoder (tensorflow_hub)
        elif callable(embedding_model) and not hasattr(embedding_model, 'vector_size'):
            query_embedding = embedding_model([query]).numpy()[0]
            product_embedding = embedding_model([product_name]).numpy()[0]
        
        # Word2Vec (gensim) - используем средний эмбеддинг слов
        elif hasattr(embedding_model, 'vector_size') or hasattr(embedding_model, 'index_to_key'):
            query_processed = preprocess_query(query)
            product_processed = preprocess_query(product_name)
            
            def get_word2vec_embedding(text, model):
                words = text.split()
                if len(words) == 0:
                    return np.zeros(model.vector_size if hasattr(model, 'vector_size') else 300)
                
                embeddings = []
                for word in words:
                    try:
                        if hasattr(model, 'index_to_key'):
                            # KeyedVectors
                            if word in model.index_to_key:
                                embeddings.append(model[word])
                        else:
                            # Word2Vec model
                            if word in model.wv:
                                embeddings.append(model.wv[word])
                    except:
                        continue
                
                if len(embeddings) == 0:
                    return np.zeros(model.vector_size if hasattr(model, 'vector_size') else 300)
                
                return np.mean(embeddings, axis=0)
            
            query_embedding = get_word2vec_embedding(query_processed, embedding_model)
            product_embedding = get_word2vec_embedding(product_processed, embedding_model)
        
        else:
            raise ValueError("Неизвестный тип модели эмбеддингов")
        
        # Вычисляем косинусное сходство
        similarity = cosine_similarity([query_embedding], [product_embedding])[0][0]
        
        # Нормализуем схожесть в вероятность [0, 1]
        # Косинусное сходство в диапазоне [-1, 1], нормализуем в [0, 1]
        probability = (similarity + 1) / 2
        
        return float(probability), float(similarity)
        
    except Exception as e:
        print(f"Ошибка вычисления эмбеддингов: {e}")
        # Fallback на TF-IDF
        try:
            return calculate_purchase_probability_tfidf(query, product_name, None, None, data if 'data' in globals() else None)
        except:
            return 0.0, 0.0

def calculate_purchase_probability_tfidf(query, product_name, tfidf_dict=None, tfidf_feat=None, data_df=None):
    """
    Fallback функция: вычисляет вероятность покупки используя только TF-IDF
    """
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.metrics.pairwise import cosine_similarity
    import numpy as np
    
    if tfidf_dict is None or tfidf_feat is None:
        if data_df is not None:
            tifd_temp = TfidfVectorizer()
            tifd_temp.fit_transform(data_df['product_name'])
            tfidf_dict = dict(zip(tifd_temp.get_feature_names_out(), list(tifd_temp.idf_)))
            tfidf_feat = tifd_temp.get_feature_names_out()
        else:
            return 0.0, 0.0
    
    query_processed = preprocess_query(query)
    product_processed = preprocess_query(product_name)
    
    # Создаем простые TF-IDF векторы
    query_words = query_processed.split()
    product_words = product_processed.split()
    
    # Вычисляем TF-IDF вручную
    def get_tfidf_vector(words, tfidf_dict, tfidf_feat):
        vec = np.zeros(len(tfidf_feat))
        word_to_idx = {word: idx for idx, word in enumerate(tfidf_feat)}
        for word in words:
            if word in word_to_idx:
                tf = words.count(word) / len(words) if len(words) > 0 else 0
                idf = tfidf_dict.get(word, 1.0)
                vec[word_to_idx[word]] = tf * idf
        return vec
    
    query_vec = get_tfidf_vector(query_words, tfidf_dict, tfidf_feat)
    product_vec = get_tfidf_vector(product_words, tfidf_dict, tfidf_feat)
    
    similarity = cosine_similarity([query_vec], [product_vec])[0][0]
    probability = (similarity + 1) / 2
    
    return float(probability), float(similarity)

print("Функции calculate_purchase_probability и get_embedding_model определены")


In [ ]:
def generate_feed_matrix(query, n_products, vectorizer, product_vectors, data_df, rows=2, cols=2, embedding_model=None, image_model=None):
    """
    Генерирует матрицу размера n * 2, где каждая ячейка содержит:
    - id: идентификатор товара
    - P_click: схожесть товара с запросом от 0 до 1 (TF-IDF)
    - P_buy: схожесть товара с запросом от 0 до 1 (sentence-transformers или другая модель)
    - P_look1: схожесть с товаром сверху по изображениям от 0 до 1
    - P_look2: схожесть с товаром слева/справа по изображениям от 0 до 1
    
    Параметры:
    -----------
    query : str
        Текстовый запрос пользователя
    n_products : int
        Количество товаров (должно быть rows * cols)
    vectorizer : TfidfVectorizer
        Векторизатор TF-IDF
    product_vectors : sparse matrix
        TF-IDF векторы товаров
    data_df : pandas.DataFrame
        DataFrame с данными о товарах
    rows : int
        Количество строк в матрице
    cols : int
        Количество колонок в матрице (обычно 2)
    embedding_model : optional
        Модель для эмбеддингов (sentence-transformers) для P_buy
    image_model : optional
        Модель VGG16 для извлечения эмбеддингов изображений
    
    Возвращает:
    -----------
    matrix : list of list of dict
        Матрица размера rows * cols, где каждый элемент - словарь с ключами:
        - 'id': str
        - 'P_click': float (0-1)
        - 'P_buy': float (0-1)
        - 'P_look1': float (0-1) или None если нет товара сверху
        - 'P_look2': float (0-1) или None если нет товара слева/справа
    """
    from sklearn.metrics.pairwise import cosine_similarity
    from sklearn.feature_extraction.text import TfidfVectorizer
    import numpy as np
    import pandas as pd
    
    # Генерируем выдачу товаров
    results = generate_feed_tfidf(query, n_products, vectorizer, product_vectors, data_df, rows=rows)
    
    # Вычисляем P_click для каждого товара (TF-IDF схожесть с запросом)
    query_processed = preprocess_query(query)
    query_vector = vectorizer.transform([query_processed])
    
    # Получаем индексы товаров в исходном DataFrame
    product_indices = [item['vector_index'] for item in results]
    product_vectors_subset = product_vectors[product_indices]
    
    # Вычисляем схожесть TF-IDF
    tfidf_similarities = cosine_similarity(query_vector, product_vectors_subset)[0]
    # Нормализуем в [0, 1]
    min_sim = tfidf_similarities.min()
    max_sim = tfidf_similarities.max()
    if max_sim - min_sim > 0:
        p_click_values = (tfidf_similarities - min_sim) / (max_sim - min_sim)
    else:
        p_click_values = np.ones_like(tfidf_similarities)
    
    # Вычисляем P_buy для каждого товара (sentence-transformers схожесть)
    if embedding_model is None:
        embedding_model = get_embedding_model()
    
    p_buy_values = []
    for item in results:
        prob, _ = calculate_purchase_probability(query, item['product_name'], embedding_model=embedding_model)
        p_buy_values.append(prob)
    
    # Вычисляем эмбеддинги изображений для всех товаров
    import tensorflow as tf
    from tensorflow.keras.applications import VGG16
    from tensorflow.keras.preprocessing import image
    from tensorflow.keras.applications.vgg16 import preprocess_input
    from tensorflow.keras.utils import get_file
    import os
    import tempfile
    
    if image_model is None:
        image_model = VGG16(weights='imagenet', include_top=False)
    
    def extract_image_embedding(img_url, model):
        """Извлекает эмбеддинг изображения"""
        try:
            if isinstance(img_url, str) and '|' in img_url:
                img_url = img_url.split('|')[0]
            
            if not img_url or pd.isna(img_url):
                return np.zeros((512,))
            
            # Создаем уникальное имя файла
            temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.jpg')
            temp_path = temp_file.name
            temp_file.close()
            
            try:
                img_path = get_file(temp_path, origin=img_url)
                img = image.load_img(img_path, target_size=(224, 224))
                img_array = image.img_to_array(img)
                img_array = np.expand_dims(img_array, axis=0)
                img_array = preprocess_input(img_array)
                features = model.predict(img_array, verbose=0)
                embedding = features.flatten()
                
                # Удаляем временный файл
                if os.path.exists(temp_path):
                    os.remove(temp_path)
                
                if np.all(embedding == 0):
                    return np.random.rand(512) * 0.01
                
                return embedding
            except Exception as e:
                if os.path.exists(temp_path):
                    os.remove(temp_path)
                return np.random.rand(512) * 0.01
        except:
            return np.random.rand(512) * 0.01
    
    # Извлекаем эмбеддинги для всех товаров
    image_embeddings = []
    for item in results:
        emb = extract_image_embedding(item['image_url'], image_model)
        image_embeddings.append(emb)
    
    image_embeddings = np.array(image_embeddings)
    
    # Вычисляем матрицу схожести изображений
    image_similarity_matrix = cosine_similarity(image_embeddings)
    # Нормализуем в [0, 1]
    min_sim = image_similarity_matrix.min()
    max_sim = image_similarity_matrix.max()
    if max_sim - min_sim > 0:
        image_similarity_normalized = (image_similarity_matrix - min_sim) / (max_sim - min_sim)
    else:
        image_similarity_normalized = image_similarity_matrix
    
    # Формируем матрицу
    matrix = []
    for row in range(rows):
        matrix_row = []
        for col in range(cols):
            idx = row * cols + col
            if idx >= len(results):
                matrix_row.append(None)
                continue
            
            item = results[idx]
            
            # P_look1: схожесть с товаром сверху по изображениям
            p_look1 = None
            if row > 0:
                up_idx = (row - 1) * cols + col
                if up_idx < len(results):
                    p_look1 = float(image_similarity_normalized[idx, up_idx])
            
            # P_look2: схожесть с товаром слева или справа по изображениям
            p_look2 = None
            if col > 0:
                # Товар слева
                left_idx = row * cols + (col - 1)
                if left_idx < len(results):
                    p_look2 = float(image_similarity_normalized[idx, left_idx])
            elif col < cols - 1:
                # Товар справа
                right_idx = row * cols + (col + 1)
                if right_idx < len(results):
                    p_look2 = float(image_similarity_normalized[idx, right_idx])
            
            matrix_row.append({
                'id': item['id'],
                'P_click': float(p_click_values[idx]),
                'P_buy': float(p_buy_values[idx]),
                'P_look1': float(p_look1) if p_look1 is not None else None,
                'P_look2': float(p_look2) if p_look2 is not None else None
            })
        matrix.append(matrix_row)
    
    return matrix

def display_feed_matrix(matrix, rows=2, cols=2):
    """
    Отображает матрицу выдачи в удобном виде
    
    Параметры:
    -----------
    matrix : list of list of dict
        Матрица от generate_feed_matrix
    rows : int
        Количество строк
    cols : int
        Количество колонок
    """
    print("=" * 100)
    print("МАТРИЦА ВЫДАЧИ ТОВАРОВ")
    print("=" * 100)
    
    for row in range(rows):
        print(f"\nСтрока {row + 1}:")
        print("-" * 100)
        for col in range(cols):
            if matrix[row][col] is None:
                print(f"  Колонка {col + 1}: [пусто]")
                continue
            
            item = matrix[row][col]
            print(f"\n  Колонка {col + 1}:")
            print(f"    ID: {item['id']}")
            print(f"    P_click (TF-IDF): {item['P_click']:.4f}")
            print(f"    P_buy (sentence-transformers): {item['P_buy']:.4f}")
            print(f"    P_look1 (сверху): {item['P_look1']:.4f}" if item['P_look1'] is not None else "    P_look1 (сверху): N/A")
            print(f"    P_look2 (слева/справа): {item['P_look2']:.4f}" if item['P_look2'] is not None else "    P_look2 (слева/справа): N/A")
        
        if row < rows - 1:
            print("\n" + "=" * 100)
    
    print("\n" + "=" * 100)
    
    # Сводная таблица
    import pandas as pd
    flat_data = []
    for row in range(rows):
        for col in range(cols):
            if matrix[row][col] is not None:
                item = matrix[row][col].copy()
                item['row'] = row + 1
                item['col'] = col + 1
                flat_data.append(item)
    
    if flat_data:
        df = pd.DataFrame(flat_data)
        print("\nСводная таблица:")
        print(df.to_string(index=False))


In [ ]:
# Пример использования: генерация матрицы выдачи
query = "women cotton dress"
n_products = 4  # 2 строки * 2 колонки
rows = 2
cols = 2

# Генерируем матрицу
matrix = generate_feed_matrix(
    query=query,
    n_products=n_products,
    vectorizer=tifd,
    product_vectors=tf,
    data_df=data,
    rows=rows,
    cols=cols
)

# Отображаем результаты
display_feed_matrix(matrix, rows=rows, cols=cols)


In [ ]:
# Пример: добавление вероятностей покупки к результатам
query = "women cotton dress"
n_products = 4
rows = 2

# Генерируем выдачу
results = generate_feed_tfidf(query, n_products, tifd, tf, data, rows=rows)

# Добавляем вероятности покупки (используя готовую модель эмбеддингов)
# Функция сама попытается загрузить sentence-transformers, Universal Sentence Encoder или Word2Vec
results_with_probs = add_purchase_probabilities_to_results(results, query)
print("Вероятности покупки вычислены")

# Отображаем результаты с вероятностями покупки
print("\n" + "=" * 80)
print("РЕЗУЛЬТАТЫ С ВЕРОЯТНОСТЯМИ ПОКУПКИ")
print("=" * 80)
for i, item in enumerate(results_with_probs):
    print(f"\nТовар {i+1}:")
    print(f"  ID: {item['id']}")
    print(f"  Название: {item['product_name']}")
    print(f"  Вероятность покупки: {item.get('purchase_probability', 0.0):.4f}")
    print(f"  Схожесть с запросом: {item.get('query_similarity', 0.0):.4f}")
    print(f"  Релевантность: {item.get('relevance_score', 0.0):.4f}")


In [25]:
# pass the for which product you want similar item and how many wants and the vector 
# representation of the all the products
RecommenderSystems(12381,5,final_counts)

ASIN : B0836ZMZC9
Brand : PRAKASAM COTTON
Product Name : prakasam cotton men kfc plain flexi size welcro pocket dhoti kfc-pln-ash
Distance : 0.0
____________________________________________________________


ASIN : B01EWZ9WVQ
Brand : PRAKASAM COTTON
Product Name : prakasam cotton men cotton dhoti
Distance : 3.4641016151377544
____________________________________________________________


ASIN : B00YUJLOQQ
Brand : Ramraj
Product Name : ramraj men cotton dhoti
Distance : 3.605551275463989
____________________________________________________________


ASIN : B07C9C6Z7J
Brand : PRAKASAM COTTON
Product Name : prakasam cotton men winner velcro pocket singl dhoti
Distance : 3.605551275463989
____________________________________________________________


ASIN : B07R25DYZN
Brand : PRAKASAM COTTON
Product Name : prakasam cotton men jari cotton dhoti
Distance : 3.605551275463989
____________________________________________________________


### Using TF-IDF

In [26]:
# compute the TF-IDF
#tfidf
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
tifd=TfidfVectorizer(ngram_range=(1,2), min_df=10)
tf=tifd.fit_transform(data['product_name'])
print("the shape of out text TFIDF vectorizer ",tf.get_shape())

the shape of out text TFIDF vectorizer  (15949, 3082)


### Similarity Based Recommender systems using Tf-IDF

In [27]:
# pass the for which product you want similar item and how many wants and the vector 
# representation of the all the products
RecommenderSystems(10101,15,tf)

ASIN : B06WGQLN97
Brand : SOFYANA
Product Name : sofyana babi gilr birthday parti dress girl pink tissu net 7-8 year
Distance : 0.0
____________________________________________________________


ASIN : B078V2W64J
Brand : MVD Fashion
Product Name : mvd fashion parti dress girl
Distance : 0.9323216101199711
____________________________________________________________


ASIN : B071NJXLHJ
Brand : ALL ABOUT PINKS
Product Name : pink dress girl birthday dress babi girl frock parti dress girl dress girl frock dress frock girl pink & white 2 3 years
Distance : 0.9927003226717918
____________________________________________________________


ASIN : B071HC9WKL
Brand : ALL ABOUT PINKS
Product Name : pink dress girl birthday dress babi girl frock parti dress girl dress girl frock dress frock girl pink & white 6 7 years
Distance : 0.9927003226717918
____________________________________________________________


ASIN : B07Z9GDP82
Brand : IAC
Product Name : iacjacketkidsb0i
Distance : 0.9999999999999998
____________________________________________________________


...

ASIN : B07H9WKQS7
Brand : ADDYVERO
Product Name : addyvero girl flounc sleev parti dress
Distance : 1.0233659601894667
____________________________________________________________


ASIN : B06Y28NFH7
Brand : ALL ABOUT PINKS
Product Name : pink dress girl birthday dress babi girl frock parti dress girl dress girl frock dress frock girl 7-8 year blue
Distance : 1.0320000887223209
____________________________________________________________


ASIN : B06Y28PN6X
Brand : ALL ABOUT PINKS
Product Name : pink dress girl birthday dress babi girl frock parti dress girl dress girl frock dress frock girl 4-5 year blue
Distance : 1.0320000887223209
____________________________________________________________


ASIN : B07F8ZLZ7D
Brand : ALL ABOUT PINKS
Product Name : pink girl pink cotton partywear dress - 2-3 years
Distance : 1.039687506158301
____________________________________________________________


ASIN : B07D17KYFS
Brand : SOFYANA
Product Name : sofyana babi girl tissu net polyest satin birthday dress sfn057peach gown
Distance : 1.054467867862887
____________________________________________________________


### Using Word2Vec

In [28]:
# load the google model
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
w2v_model=KeyedVectors.load_word2vec_format('/kaggle/input/gensimmodel/GoogleNews-vectors-negative300.bin', binary=True)

In [29]:
# average Word2Vec
# compute average word2vec for each Product Title
# words which are present in the w2vec model
w2v_words = list(w2v_model.index_to_key)
sent_vectors = []; # the avg-w2v for each product title is stored in this list
for sent in tqdm(data['product_name']): # for each review/sentence
    # length of word vector needed
    sent_vec = np.zeros(300) 
    cnt_words =0; # num of words with a valid vector in the product title
    # for each word in title
    for word in sent: # for each word in a review/sentence
        if word in w2v_words:
            vec = w2v_model[word]
            sent_vec += vec
            cnt_words += 1
    if cnt_words != 0:
        sent_vec /= cnt_words
    sent_vectors.append(sent_vec)
print(len(sent_vectors))
print(len(sent_vectors[0]))

100%|██████████| 15949/15949 [3:16:16<00:00,  1.35it/s]  

15949
300


In [30]:
# store the vector for product title vector in the pikle file so no need to run the above
# code again to generate it
import pickle
file_path = '/kaggle/working/product.pkl'
with open(file_path, 'wb') as file:
    pickle.dump(sent_vectors, file)

In [32]:
# Read the pickle file again
file_path = '/kaggle/working/product.pkl'
with open(file_path, 'rb') as file:
    product_vector = pickle.load(file)

In [33]:
pv = np.array(product_vector)

#### Remmendation using the word2Vec for Text to vector and ecludian distance as similarity

In [35]:
RecommenderSystems(4310,5,pv)

ASIN : B003PKPKES
Brand : Chevrolet
Product Name : chevrolet corvett c6 flex black basebal cap
Distance : 0.0
____________________________________________________________


ASIN : B003PKVT5M
Brand : Chevrolet
Product Name : corvett c6 flex red basebal cap
Distance : 0.1990861492919667
____________________________________________________________


ASIN : B0154REIJW
Brand : Verceys
Product Name : vercey black white casual belt women - pack 2
Distance : 0.2772062629528931
____________________________________________________________


ASIN : B019ZJ1WWG
Brand : Black Collection
Product Name : black collect solid men v-neck strecthabl shirt
Distance : 0.317829593468222
____________________________________________________________


ASIN : B07T6KYG37
Brand : Calvus
Product Name : calvus men black windcheat hidden collar pocket cap
Distance : 0.3241353139908091
____________________________________________________________


### weighthed TF-IDF word2vec Based Recommendation System

In [36]:
tifd=TfidfVectorizer()
tifd.fit_transform(data['product_name'])
# we are converting a dictionary with word as a key, and the idf as a value
dictionary = dict(zip(tifd.get_feature_names_out(), list(tifd.idf_)))
# TF-IDF weighted Word2Vec
tfidf_feat = tifd.get_feature_names_out() # tfidf words/col-names
# final_tf_idf is the sparse matrix with row= sentence, col=word and cell_val = tfidf
tfidf_sent_vectors = []; # the tfidf-w2v for each product is stored in this list
row=0;
for sent in tqdm(data['product_name']): # for each products
    sent_vec = np.zeros(300) # as word vectors are of zero length
    weight_sum =0; # num of words with a valid vector in the products
    for word in sent.split(): # for each word in a product
        if word in w2v_words and word in tfidf_feat:
            vec = w2v_model[word]
#             tf_idf = tf_idf_matrix[row, tfidf_feat.index(word)]
            # to reduce the computation we are 
            # dictionary[word] = idf value of word in whole courpus
            # sent.count(word) = tf valeus of word in this review
            tf_idf = dictionary[word]*(sent.count(word)/len(sent))
            sent_vec += (vec * tf_idf)
            weight_sum += tf_idf
    if weight_sum != 0:
        sent_vec /= weight_sum
    tfidf_sent_vectors.append(sent_vec)
    row += 1

100%|██████████| 15949/15949 [46:47<00:00,  5.68it/s] 


In [37]:
# store the vector for product title vector in the pikle file so no need to run the above
# code again to generate it
file_path = '/kaggle/working/product_tf.pkl'
with open(file_path, 'wb') as file:
    pickle.dump(tfidf_sent_vectors, file)

In [38]:
RecommenderSystems(100,5,tfidf_sent_vectors)

ASIN : B07KT61N7D
Brand : JAIPUR ATTIRE
Product Name : jaipur attir women cotton flare kurti
Distance : 0.0
____________________________________________________________


ASIN : B07W2Z1JRV
Brand : VEDANA
Product Name : vedana cotton print flare kurti pink
Distance : 0.9250530634480277
____________________________________________________________


ASIN : B07WGQTBP9
Brand : R.R.TEXTILES
Product Name : rrtextil red flare rayon kurti women design kurti girl
Distance : 0.9532673267324706
____________________________________________________________


ASIN : B07MH5BWDY
Brand : PALSIYA PRESENTS
Product Name : palsiya present women rayon navyblu flare print kurti
Distance : 0.9555220972030847
____________________________________________________________


ASIN : B0765ZLGF1
Brand : JAIPUR ATTIRE
Product Name : jaipur attir women cotton slub straight kurti
Distance : 0.9572684858360978
____________________________________________________________


### Also Add the Brand based Recommendation along with the title of product

In [39]:
#  as we number of unique brands are 5403 we can use the One Hot Encoding to Encode 
# all these brand
# Encode all the brand
brand_vec = pd.get_dummies(data['brand'], prefix='brand')

In [44]:
# Now We have brand and title both to compute or find the recommendation 
# most often we need biased based Recommendation 
# change the model with adding the bias as well
def weightedRecommenderSystems(p_id,numProduct,vectors,category_weight):
    """
    p_id is the index from data that  is preprocessed
    numProduct of wanted to recommeded
    category on what feature concat to create
    category weights what weight of these category can be
    vectors contains set of vector for each of category based on their weight
    """
    # create an weighted Vectors
    for i in range(len(vectors)):
        vectors[i]=np.array(vectors[i])*category_weight[i]
    vectors=np.concatenate(vectors, axis=1)
    # Compute the Pairwise Distances
    dist=pairwise_distances(vectors,vectors[p_id].reshape(1,-1))
    # sort them based the distance computed
    indices=np.argsort(dist.flatten())[0:numProduct]
    # this will store distances between the product_titles
    p_dist=np.sort(dist.flatten())[0:numProduct]
    # take product details for these indices
    similar_product=list(data.index[indices])
    for i in range(len(indices)):
        # generate the heatmap as well
        display(Image(url=data['medium'].loc[similar_product[i]],width=100, height=100))
        print('ASIN :',data['asin'].loc[similar_product[i]])
        print('Brand :',data['brand'].loc[similar_product[i]])
        print('Product Name :',data['product_name'].loc[similar_product[i]])
        print('Distance :',p_dist[i])
        print('_'*60)

In [48]:
# giving more weights to the title then brands 
# based on weights and category vector passed will return the Recommended Products
weightedRecommenderSystems(2215,10,[tfidf_sent_vectors,brand_vec],[1,.5])

ASIN : B07SJZKS6Z
Brand : Varanga
Product Name : varanga women cotton print a-lin handloom dress off white  xxl
Distance : 0.0
____________________________________________________________


ASIN : B07GXYQ7K7
Brand : RLB Fashion
Product Name : rlb fashion women cotton silk handloom dhakai jamdani sare rlb-00547 pink free size
Distance : 0.9966957731430856
____________________________________________________________


ASIN : B07RKJ8K3K
Brand : RLB Fashion
Product Name : rlb fashion women cotton silk handloom dhakai jamdani sare white red
Distance : 1.011840379029101
____________________________________________________________


ASIN : B07BVCNS9C
Brand : RLB Fashion
Product Name : rlb fashion women cotton silk handloom dhakai jamdani sare rlb-00435 grey free size
Distance : 1.0298298867020217
____________________________________________________________


ASIN : B07YWNP586
Brand : Genric
Product Name : women chandana ikkat pure handloom pochamp ikat cotton unstitch dress materi red free size
Distance : 1.0569849069345518
____________________________________________________________


ASIN : B07YWN727W
Brand : Genric
Product Name : women chandana ikkat handloom pochamp ikat cotton unstitch dress materi blue free size
Distance : 1.0626977478026942
____________________________________________________________


ASIN : B07TMPFWF9
Brand : Handloom Palace
Product Name : handloom palac hand block cotton floral print veget color run dress materi unstitch kantha fabric
Distance : 1.0973368616899726
____________________________________________________________


ASIN : B07V4F6PBF
Brand : T.J. SAREES
Product Name : tj sare women handloom cotton silk hand print sare black & red
Distance : 1.1070599712548617
____________________________________________________________


ASIN : B07Q5XXFMG
Brand : WoodenTant
Product Name : woodent red cotton silk soft dhakai jamdani handloom sare
Distance : 1.140453280100687
____________________________________________________________


ASIN : B07FGG9475
Brand : QueenShield
Product Name : queenshield women casual & formal handloom cotton kurti plus small size â€¦
Distance : 1.1404963390440899
____________________________________________________________


### Recommendation Using the Image as item based similarity

In [136]:
# step1 : Extract the Feature from image using an pretrained model of Vgg16
# Extracting feature from images
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.utils import get_file
import numpy as np
# Using the Tensorflow to extract freatue
# using the pretrained model of VGG16 which is an cnn for image dataset 
# extracting by removing the last layer of model

#step1.1 Loading the pre-trained VGG16 model
model = VGG16(weights='imagenet', include_top=False)

def extract_features(img_path):
    # download the image store it temerarly
    img_path = get_file('image.jpg', origin=img_path)
    #  resizing all images in 244*244 size
    img = image.load_img(img_path, target_size=(224, 224))
    # convert the image to array
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    # preprocess the array as the input required for the vgg16 model
    # using the function preprocess_input
    img_array = preprocess_input(img_array)
    features = model.predict(img_array)
    # return the feature extracted
    return features.flatten()

# for each image find its feature extract from above funtion and store them as a list
img_features = data['medium'].apply(extract_features)
print(len(img_features))


1/1 [==============================] - 0s 483ms/step
[[[[0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
    0.0000000e+00 0.0000000e+00]
   [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
    0.0000000e+00 0.0000000e+00]
   [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
    0.0000000e+00 0.0000000e+00]
   ...
   [9.2670784e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
    3.6524075e+01 0.0000000e+00]
   [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
    4.3967037e+01 0.0000000e+00]
   [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
    0.0000000e+00 0.0000000e+00]]

  [[0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
    0.0000000e+00 0.0000000e+00]
   [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
    0.0000000e+00 0.0000000e+00]
   [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
    0.0000000e+00 0.0000000e+00]
   ...
   [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.000000

### Recommendation using the Image as a input 

In [148]:
# store the extracted feature vector in pickle file
file_path = '/kaggle/working/product_img.pkl'
with open(file_path, 'wb') as file:
    pickle.dump(img_features, file)

In [157]:
## we can generate as many as product we want as we can compare with different 
# model by having the all brand,image,title in considration the model 
# recommends perfectly
weightedRecommenderSystems(4310,10,[tfidf_sent_vectors,brand_vec,features],[1,1,1])

ASIN : B003PKPKES
Brand : Chevrolet
Product Name : chevrolet corvett c6 flex black basebal cap
Distance : 3.0517578125e-05
____________________________________________________________


ASIN : B003PKVT5M
Brand : Chevrolet
Product Name : corvett c6 flex red basebal cap
Distance : 0.5220768925258924
____________________________________________________________


ASIN : B0147NMIL4
Brand : Chevrolet
Product Name : hot rod plus chevrolet corvett c7 logo gray basebal hat
Distance : 1.140350975759019
____________________________________________________________


ASIN : B003MRDHNK
Brand : Chevrolet
Product Name : chevrolet camaro ralli stripe yellow basebal cap
Distance : 1.187063581454872
____________________________________________________________


ASIN : B004X8XHZ0
Brand : Chevrolet
Product Name : corvett c3 cross flag black basebal cap
Distance : 1.3958643297026474
____________________________________________________________


ASIN : B003FPA6MY
Brand : Chevrolet
Product Name : camaro z28 liquid metal basebal hat
Distance : 1.4469381076354135
____________________________________________________________


ASIN : B01MUHMHZH
Brand : Chevrolet
Product Name : c7 corvett stingray hat - black embroid gestur logo
Distance : 1.7522674212709526
____________________________________________________________


ASIN : B0010ZVYPO
Brand : High-End Motorsports
Product Name : high-end motorsport c6 cottontwil black hat compat 2005-2013 chevrolet corvett
Distance : 1.7986300956911845
____________________________________________________________


ASIN : B00KKXLKUQ
Brand : Joe's USA
Product Name : premium flex fit hat - high perform cool & dri basebal cap 7 color
Distance : 1.8146561207771337
____________________________________________________________


ASIN : B07T1B89LV
Brand : Alcove
Product Name : alcov unisex black red navyblu white ny half meshhalf net basebal fabric cotton cap combo 4 curv visor freesizeadjust
Distance : 1.8199569236930293
____________________________________________________________
